In [1]:
!python interpretar_fold_paper.py shap \
  --run_dir ./resultados_13_paper \
  --fold 1 \
  --clf mlp \
  --global_tensor_path /home/diego/Escritorio/limpio/AAL3_dynamicROIs_fmri_tensor_NeuroEnhanced_v6.5.17_AAL3_131ROIs_OMST_GCE_Signed_GrangerLag1_ChNorm_ROIreorderedYeo17_ParallelTuned/GLOBAL_TENSOR_from_AAL3_dynamicROIs_fmri_tensor_NeuroEnhanced_v6.5.17_AAL3_131ROIs_OMST_GCE_Signed_GrangerLag1_ChNorm_ROIreorderedYeo17_ParallelTuned.npz \
  --metadata_path /home/diego/Escritorio/limpio/SubjectsData_AAL3_procesado.csv \
  --channels_to_use 1 2 5 \
  --latent_dim 512 \
  --latent_features_type mu \
  --metadata_features Age Sex \
  --num_conv_layers_encoder 4 \
  --decoder_type convtranspose \
  --dropout_rate_vae 0.2 \
  --intermediate_fc_dim_vae quarter \
  --vae_final_activation tanh \
  --kernel_nsamples 800



[INFO] [SHAP] fold=1 clf=mlp
[WARNING] label_mapping.json no encontrado; se asume CN=0 / AD=1
[INFO] [SHAP] X_raw (test) shape=(37, 514) (latentes + 2 metadatos)
[INFO] [SHAP] Latentes detectadas en procesado: 512 / 514
[WARNING] [SHAP] No hay RAW; uso el procesado: shap_background_data_mlp.joblib
[WARNING] [SHAP] Usando KernelExplainer (puede ser lento).
[INFO] [SHAP] Resumiendo el background de 100 muestras a 50 centroides para KernelExplainer.
100%|███████████████████████████████████████████| 37/37 [00:59<00:00,  1.62s/it]
[INFO] [SHAP] Pack guardado: resultados_13_paper/fold_1/interpretability_shap/shap_pack_mlp.joblib


In [2]:
!python interpretar_fold_paper.py saliency \
  --run_dir ./resultados_13_paper \
  --fold 1 \
  --clf mlp \
  --global_tensor_path /home/diego/Escritorio/limpio/AAL3_dynamicROIs_fmri_tensor_NeuroEnhanced_v6.5.17_AAL3_131ROIs_OMST_GCE_Signed_GrangerLag1_ChNorm_ROIreorderedYeo17_ParallelTuned/GLOBAL_TENSOR_from_AAL3_dynamicROIs_fmri_tensor_NeuroEnhanced_v6.5.17_AAL3_131ROIs_OMST_GCE_Signed_GrangerLag1_ChNorm_ROIreorderedYeo17_ParallelTuned.npz \
  --metadata_path /home/diego/Escritorio/limpio/SubjectsData_AAL3_procesado.csv \
  --roi_annotation_path "/home/diego/Escritorio/limpio/roi_info_master.csv" \
  --channels_to_use 1 2 5 \
  --latent_dim 512 \
  --latent_features_type mu \
  --metadata_features Age Sex \
  --num_conv_layers_encoder 4 \
  --decoder_type convtranspose \
  --dropout_rate_vae 0.2 \
  --intermediate_fc_dim_vae quarter \
  --vae_final_activation tanh \
  --saliency_method "integrated_gradients" \
  --top_k 50

[INFO] [SALIENCY] fold=1 clf=mlp  (pack cargado: shap_pack_mlp.joblib)
[INFO] Usando ROI order de resultados_13_paper/roi_order_131.joblib.
[INFO] Cargado fichero de anotaciones: /home/diego/Escritorio/limpio/roi_info_master.csv
[INFO] [SALIENCY] 50 latentes ponderadas. Ejemplo:
 latent_idx    weight  importance    feature
        479  0.036931    0.005941 latent_479
        215  0.036441    0.005862 latent_215
         15  0.036379    0.005852  latent_15
         67 -0.031471   -0.005063  latent_67
        210  0.030629    0.004927 latent_210
[INFO] [SALIENCY] Sujetos AD=19  CN=18
[INFO] [SALIENCY] Usando método de saliencia: integrated_gradients
[INFO] [SALIENCY] Ranking de conexiones ANOTADO guardado: resultados_13_paper/fold_1/interpretability_mlp/ranking_conexiones_ANOTADO_integrated_gradients_top50.csv
[INFO] Top 20 conexiones anotadas:
      Rank src_AAL3_Name dst_AAL3_Name  Saliency_Score          src_Refined_Network          dst_Refined_Network
1816     1          SMAL        

In [23]:
# === Importaciones y configuración inicial ===
import matplotlib as mpl
from tqdm.autonotebook import tqdm as notebook_tqdm
from pathlib import Path
import joblib, numpy as np, pandas as pd
import matplotlib.pyplot as plt
import shap, re

# === Función de ayuda para ajustar la opacidad ===
def _ajustar_opacidad_violin(ax=None, alpha=0.35, subir_puntos=True):
    """
    Baja la opacidad de los violines creados por shap.summary_plot(plot_type='violin').
    alpha: 0 (transparente) ... 1 (opaco)
    subir_puntos: si hay puntos (beeswarm), los pone por encima con un borde.
    """
    ax = ax or plt.gca()
    # Ajusta la opacidad de los violines (PolyCollection)
    for pc in ax.findobj(mpl.collections.PolyCollection):
        pc.set_alpha(alpha)
    # Ajusta la opacidad de otros patches (por si acaso)
    for p in ax.patches:
        try: p.set_alpha(alpha)
        except Exception: pass
    if subir_puntos:
        # Pone los puntos del beeswarm (PathCollection) por delante
        for sc in ax.findobj(mpl.collections.PathCollection):
            sc.set_zorder(3)
            # Opcional: añade un borde blanco para que destaquen más
            sc.set_edgecolor("white")
            sc.set_linewidth(0.4)

# === Carga de datos ===
RUN_DIR = Path("./resultados_13_paper")
FOLD    = 1
CLF     = "mlp"

pack = joblib.load(RUN_DIR / f"fold_{FOLD}/interpretability_shap/shap_pack_{CLF}.joblib")
shap_vals = pack["shap_values"]            # (N,F) para clase positiva
Xtest     = pack["X_test"]                 # DataFrame post-preproc
featnames = list(map(str, pack["feature_names"]))

# Máscaras robustas para seleccionar features
latent_mask = np.array([re.search(r'(?:^|__)latent_\d+\b', c) is not None for c in featnames])
age_mask    = np.array([re.search(r'(?:^|__)Age\b', c) is not None for c in featnames])

# === Gráfico combinado para 'Age' (Violin + Beeswarm) ===
if age_mask.any():
    plt.figure(figsize=(6, 2.8))
    # 1. Dibuja el violín
    shap.summary_plot(shap_vals[:, age_mask], Xtest.loc[:, age_mask],
                      plot_type="violin", show=False, max_display=1)

    # 2. 📌 Ajusta la opacidad del violín
    _ajustar_opacidad_violin(alpha=0.35)

    # 3. 📌 Añade los puntos (beeswarm) encima
    ax = plt.gca()
    vals = shap_vals[:, age_mask].ravel()
    # Un poco de jitter vertical para que los puntos no se superpongan
    y_jitter = 0.03 * np.random.randn(len(vals))
    ax.scatter(vals, y_jitter, s=14, zorder=3, c="#1f77b4",
               edgecolors="white", linewidths=0.4) # Usamos zorder=3 para ponerlos al frente

    plt.title(f"SHAP – Age (Violin + Beeswarm) · Fold {FOLD} · {CLF.upper()}")
    plt.tight_layout()
    # Guardamos con un nombre que refleje el nuevo contenido
    plt.savefig(RUN_DIR / f"fold_{FOLD}/interpretability_shap/shap_age_combined.png", dpi=150)
    plt.close()
else:
    print("⚠️ 'Age' no está entre las columnas de X_test (puede que el selector la haya removido).")

# === Gráficos para features latentes (Violin y Beeswarm separados) ===
if latent_mask.any():
    Xlat = Xtest.loc[:, latent_mask]
    S_lat = shap_vals[:, latent_mask]

    # --- Violín semitransparente para latentes ---
    plt.figure(figsize=(10, 8))
    shap.summary_plot(S_lat, Xlat, plot_type="violin", show=False, max_display=20)
    # 📌 Aplicamos el helper para bajar la opacidad de los violines
    _ajustar_opacidad_violin(alpha=0.35)
    plt.title(f"SHAP – Latent features (violin) · Fold {FOLD} · {CLF.upper()}")
    plt.tight_layout()
    plt.savefig(RUN_DIR / f"fold_{FOLD}/interpretability_shap/shap_latents_violin.png", dpi=150)
    plt.close()

    # --- Beeswarm con puntos resaltados para latentes ---
    plt.figure(figsize=(10, 8))
    shap.summary_plot(S_lat, Xlat, show=False, max_display=20)  # beeswarm por defecto
    # 📌 Aplicamos el helper para subir los puntos y añadirles un borde
    _ajustar_opacidad_violin(alpha=1.0, subir_puntos=True)
    plt.title(f"SHAP – Latent features (beeswarm) · Fold {FOLD} · {CLF.upper()}")
    plt.tight_layout()
    plt.savefig(RUN_DIR / f"fold_{FOLD}/interpretability_shap/shap_latents_beeswarm.png", dpi=150)
    plt.close()
else:
    print("⚠️ No se detectaron columnas latentes ('latent_*').")

print("✅ Gráficos SHAP generados y guardados con éxito.")

✅ Gráficos SHAP generados y guardados con éxito.
